# Bedrock Knowledge Base Retrieval and Generation with Reranking

The Rerank API in Amazon Bedrock is a new feature that improves the accuracy and relevance of responses in Retrieval-Augmented Generation (RAG) applications. It supports reranker models that rank a set of retrieved documents based on their relevance to a user's query, helping to prioritize the most relevant content for response generation.

## Key features and use cases:

1. **Enhancing RAG applications**: The Rerank API addresses challenges in semantic search, particularly with complex or ambiguous queries. For example, it can help a customer service chatbot focus on return policies rather than shipping guidelines when asked about returning an online purchase.

2. **Improving search relevance**: It enables developers to significantly enhance their search relevance and content ranking capabilities, making enterprise-grade search technology more accessible.

3. **Optimizing context window usage**: By ensuring the most useful information is sent to the foundation model, it potentially reduces costs and improves response accuracy.

4. **Flexible integration**: The Rerank API can be used independently to rerank documents even if you're not using Amazon Bedrock Knowledge Bases.

5. **Multiple model support**: At launch, it supports Amazon Rerank 1.0 and Cohere Rerank 3.5 models.

6. **Customizable configurations**: Developers can specify additional model configurations as key-value pairs for more tailored reranking.

The Rerank API is available in select AWS Regions, including US West (Oregon), Canada (Central), Europe (Frankfurt), and Asia Pacific (Tokyo). It can be integrated into existing systems at scale, whether keyword-based or semantic, through a single API call in Amazon Bedrock.


![Reranking](./reranking.png)

## 1: Import Required Functions

In [ ]:
# Import necessary functions from advanced_rag_utils
from advanced_rag_utils import (
    load_variables,
    setup_bedrock_client,
    get_value_by_key_path,
    invoke_bedrock_converse,
    search_kb_simple,
    rerank_results,
    search_rerank_combine
)

# Standard imports
import boto3
import json

## 2: Load Configuration Variables

In [ ]:
# Load the configuration variables
variables = load_variables("../variables.json")
variables

## 3: Define ARN and Configuration Details

In [ ]:
# Setting up configuration for Bedrock
region_name = variables['regionName'] 
account_number = variables['accountNumber']
knowledge_base_id = variables['kbSemanticChunk']
model_id = 'us.amazon.nova-pro-v1:0' 

# Define ARNs (Amazon Resource Names) for the model
model_arn = f"arn:aws:bedrock:us-west-2:{account_number}:inference-profile/{model_id}"
rerank_model_arn = "arn:aws:bedrock:us-west-2::foundation-model/cohere.rerank-v3-5:0"

## 4: Initialize Bedrock Clients

In [ ]:
# Initialize the Bedrock clients
bedrock_agent_client = setup_bedrock_client(region_name)
bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=region_name)

## 5: Get Initial Results from KB (Without Reranking)

In [ ]:
# Define the query
query = "What is red teaming? How can it be used with text to SQL?"

# Search the knowledge base without reranking
number_of_results = 5
original_kb_results = search_kb_simple(
    query=query,
    knowledge_base_id=knowledge_base_id,
    bedrock_client=bedrock_agent_client,
    num_results=number_of_results,
    region_name=region_name
)

# Combine results into a context string
kb_context = '\n\n'.join(original_kb_results)

## 6: Get Response from LLM (Without Reranking)
We will use the results we receive from Knowledge Base (KB) as-is. We are not doing any reranking yet.
We will send the context from KB and the user query to the LLM.

In [ ]:
# Create the system prompt
system_prompt = f"""
Please use the context below to respond to the question. 
If you have enough information to answer the question, please explain the reasoning behind the response.
If you do not have enough information to answer the question, please don't guess. Instead, just say I don't know with the reason.
CONTEXT:
{kb_context}
"""

# Get response from the LLM
answer, response = invoke_bedrock_converse(
    system_prompt=system_prompt,
    user_prompt=query,
    model_id=model_id,
    bedrock_client=bedrock_runtime_client,
    region_name=region_name
)

print("Response without reranking:")
print(answer)

## 7: Get More Results and Apply Reranking

In [ ]:
# Get more results initially, then we'll rerank them
number_of_results = 20
original_kb_results = search_kb_simple(
    query=query,
    knowledge_base_id=knowledge_base_id,
    bedrock_client=bedrock_agent_client,
    num_results=number_of_results,
    region_name=region_name
)

In [ ]:
# Rerank the results
reranked_result_count = 5
reranked_json = rerank_results(
    query=query,
    documents=original_kb_results,
    rerank_model_arn=rerank_model_arn,
    bedrock_client=bedrock_agent_client,
    reranked_result_count=reranked_result_count,
    region_name=region_name
)

In [ ]:
# Combine reranked results into a context string
kb_context = ""
for result in reranked_json['reranked_results']:
    kb_context += result['text'] + "\n\n"

## 8: Get Response from LLM (With Reranking)
Now we'll send the reranked context to the LLM and see how the response improves.

In [ ]:
# Create the system prompt with reranked context
system_prompt = f"""
Please use the context below to respond to the question. 
If you have enough information to answer the question, please explain the reasoning behind the response.
If you do not have enough information to answer the question, please don't guess. Instead, just say I don't know with the reason.
CONTEXT:
{kb_context}
"""

# Get response from the LLM with reranked context
answer, result = invoke_bedrock_converse(
    system_prompt=system_prompt,
    user_prompt=query,
    model_id=model_id,
    bedrock_client=bedrock_runtime_client,
    region_name=region_name
)

print("Response with reranking:")
print(answer)

## 9: View Response Metadata (Tokens, Latency, etc.)

In [ ]:
# View detailed response metadata
print(json.dumps(result, indent=2))

## 10: Alternative Approach - All-in-One Function
This alternative approach uses a single function to handle search, reranking, and combining results.

In [ ]:
# Use the comprehensive function to search, rerank, and combine
query = "What is red teaming? How can it be used with text to SQL?"

# Get combined context and reranking details
combined_context, reranked_details = search_rerank_combine(
    query=query,
    knowledge_base_id=knowledge_base_id,
    rerank_model_arn=rerank_model_arn,
    bedrock_client=bedrock_agent_client,
    initial_result_count=20,
    reranked_result_count=5,
    region_name=region_name
)

# Create the system prompt
system_prompt = f"""
Please use the context below to respond to the question. 
If you have enough information to answer the question, please explain the reasoning behind the response.
If you do not have enough information to answer the question, please don't guess. Instead, just say I don't know with the reason.
CONTEXT:
{combined_context}
"""

# Get response from the LLM
answer, result = invoke_bedrock_converse(
    system_prompt=system_prompt,
    user_prompt=query,
    model_id=model_id,
    bedrock_client=bedrock_runtime_client,
    region_name=region_name
)

print("Response using all-in-one function:")
print(answer)